In [8]:
import folium
import requests
import webbrowser
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from tqdm import tqdm
from pyproj import Transformer
from shapely.geometry import Polygon
from decimal import Decimal
from dataclasses import dataclass
from typing import List, Dict, Tuple, Union
from IPython.display import display, HTML, IFrame

In [9]:
def _format(coords: List[Dict]) -> List[Tuple]:
    return [(coordinates['lat'], coordinates['lon']) for coordinates in coords]
    
    
def calculate_surface_area(coordinates: List[Dict]) -> Decimal:
    coordinates = _format(coordinates)

    # Initialize projection system
    # EPSG:4326 is the WGS84 coordinate system (lat/lon)
    # EPSG:3857 is a common web mercator projection system (meters)
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857")

    # Transform coordinates from lat/lon to mercator projection
    projected_coordinates = [
        transformer.transform(lon, lat)
        for lat, lon in coordinates
    ]
    
    # Create a polygon using the projected coordinates
    polygon = Polygon(projected_coordinates)

    # Calculate the area in square meters
    area_sq_meters = polygon.area
    
    return area_sq_meters

In [10]:
def scan_osm(city: str, radius: int) -> Dict[str, Dict[str, Union[Dict,List]]]:
    
    # Call OSM API to scan and return suitable locations
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
        [out:json];
        area["name"="{city}"]->.city;

        (
          node["railway"="station"](area.city);
          node["amenity"="school"](area.city);
          node["amenity"="hospital"](area.city);
        )->.zones;

        (
          way["landuse"="brownfield"](around.zones:{str(radius)});
          way["landuse"="greenfield"](around.zones:{str(radius)});
          way["landuse"="meadow"](around.zones:{str(radius)});
        );

        out geom;
    """

    response = requests.get(url=overpass_url,
                            params={'data': overpass_query})

    data = response.json()['elements']
    
    return {
        row['id']: {
            'tags': row['tags'],
            'geometry': row['geometry']
        } for row in data
    }

In [15]:
def attach_tags(tags: Dict) -> str:
    popup_tags = f"""
    <div style="font-family: Arial, sans-serif; color: #333; background-color: #f9f9f9; padding: 10px; border-radius: 8px; box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.1);">
        <p style="font-size: 18px; margin: 0; padding: 0;">
            <strong>Landuse:</strong> {tags['landuse']}<br>
            <strong>Area:</strong> {tags['area']}
        </p>
    </div>
    """
    
    return popup_tags


@dataclass
class MapType:
    type: str
    base: str

map_types = {
    '3d': MapType(type='3d', base='base_3d.html'),
    'satellite': MapType(type='satellite', base='base_satellite.html')
}


def generate_map(map_type: MapType,
                 id: str,
                 center: str,
                 polygon: str) -> str:
    # Read the content of the file
    with open(f'{map_type.base}', 'r') as file:
        file_content = file.read()  # Read the entire content of the file
        
    # Replace the old string with the new string
    updated_content = file_content.replace('COORDINATES', center).replace('POLYGON', polygon)
    
    # Write the updated content back to the file
    generated_file_name = f'map_{map_type.type}_{id}.html'
    with open(f'{generated_file_name}', 'w') as file:
        file.write(updated_content)  # Write the modified content back to the file
        file.flush()
    
    return generated_file_name

        
def generate_additional_maps(id: str, coordinates: List):
    center = f'{str(coordinates[0][1])}, {str(coordinates[0][0])}'
    polygon = str([[lon, lat] for lat, lon in coordinates])
    
    filename_3d = generate_map(map_types['3d'], id, center, polygon)
    filename_sat = generate_map(map_types['satellite'], id, center, polygon)
    
    return filename_3d, filename_sat

In [12]:
def filter_map_data(map_data: Dict) -> Dict:
    filtered = {}

    with tqdm(total=len(map_data)) as pbar:
        for id, meta in map_data.items():

            area = calculate_surface_area(meta['geometry'])

            if area > 1500:
                meta['tags']['area'] = f"{area:.2f}sqm"
                filtered[id] = {
                    'tags': meta['tags'],
                    'geometry': [(i['lat'], i['lon']) for i in meta['geometry']]
                }

            pbar.update(1)
    
    return filtered


def generate_main_map(coordinates: List, map_data: Dict) -> folium.Map:
    m = folium.Map(location=coordinates, zoom_start=13)

    for id, meta in map_data.items():
        popup_tags = attach_tags(meta['tags'])

        # Use the base html to create urban 3D map
        filename_3d, filename_sat = generate_additional_maps(id, meta['geometry'])

        # Create a popup with increased font size using HTML
        popup_html = popup_tags + f"""
        <div style="margin-top: 10px;">
            <a href="{filename_3d}" target="_blank" style="display: inline-block; padding: 10px 20px; margin-right: 10px; background-color: #3498db; color: white; text-decoration: none; border-radius: 5px; font-size: 14px; font-weight: bold; box-shadow: 1px 1px 5px rgba(0, 0, 0, 0.1);">3D Map</a>
            <a href="{filename_sat}" target="_blank" style="display: inline-block; padding: 10px 20px; background-color: #2ecc71; color: white; text-decoration: none; border-radius: 5px; font-size: 14px; font-weight: bold; box-shadow: 1px 1px 5px rgba(0, 0, 0, 0.1);">Satellite</a>
        </div>
        """

        popup = folium.Popup(popup_html, max_width=300)

        # Create a polygon using the lat/lon coordinates directly (no projection needed here)
        folium.Polygon(
            locations=[(lat, lon) for lat, lon in meta['geometry']],
            color='blue',
            fill=True,
            fill_opacity=0.3,
            popup=popup
        ).add_to(m)

    display(m)


@dataclass
class City:
    name: str
    coordinates: List[Decimal]
        
cities = {
    'London': City(name="London", coordinates=[51.509865, -0.118092]),
    'Leeds': City(name="Leeds", coordinates=[53.801277, -1.548567]),
    'Manchester': City(name="Manchester", coordinates=[53.483959, -2.244644]),
}


def analyse(city: City, radius: int):
    map_data = scan_osm(city.name, radius)
    map_data = filter_map_data(map_data)
    
    generate_main_map(city.coordinates, map_data)

In [14]:
# Create Dropdowns
city_dropdown = widgets.Dropdown(
    options=cities.keys(),
    value='London',
    description='City:',
    disabled=False,
)

radius_input = widgets.Text(
    value='500',
    description='Radius (m):',
    layout=widgets.Layout(width='31.5%', padding="5px"),
    style={'description_width': 'initial'}
)

# Create a Submit Button
submit_button = widgets.Button(
    description="Submit",
    button_style='success'  # 'success' (green), 'info', 'warning', 'danger' or ''
)

# Main function that will be called on Submit
def main(city, radius):
    try:
        radius = float(radius)  # Validate if radius can be converted to a float
        analyse(city, radius)
    except ValueError:
        print("Please enter a valid numeric radius.")

# Define the function to be triggered when button is clicked
def on_submit_button_clicked(b):
    # Fetch the selected values from the dropdowns
    selected_city = cities[city_dropdown.value]
    selected_radius = radius_input.value
    main(selected_city, selected_radius)

# Link the button click event to the function
submit_button.on_click(on_submit_button_clicked)

# Display the dropdowns and button
display(city_dropdown, radius_input, submit_button)

Dropdown(description='City:', options=('London', 'Leeds', 'Manchester'), value='London')

Text(value='500', description='Radius (m):', layout=Layout(padding='5px', width='31.5%'), style=TextStyle(desc…

Button(button_style='success', description='Submit', style=ButtonStyle())

100%|██████████| 41/41 [00:00<00:00, 284.55it/s]
